<h2> 1. Import libraries </h2>

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import yfinance as yf

from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import minimize

from Data_Retriever import DataRetriever as DatRet
from Markowitz_PT import MarkowitzPT as MPT
from BHB_Analyzer import BHBAnalyzer as BHBA

--Frequency BHB analysis performed successfully--


<h2> 2. Define experimental variables </h2>

In [2]:
# Define number of trades
trading_n = 10

In [3]:
# Defining stock pool
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", ]
ticker_df["Seafood (food)"] = ["ORK.OL", "LSG.OL"]

<h2> 3. Retrieve data </h2>

In [4]:
# Retrieve data from yf API: y-m-d
data = DatRet(ticker_df, "2012-10-01", "2025-02-18")

# In function below, set log=True to check for data availability
data.retrieve_data()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


'--Data retrieved successfully--'

<h2> 4. Frequency trading using MPT </h2>

In [12]:
# Generate benchmark weights thorugh MPT using Sharpe ratio
benchmark = MPT(data.returns, 10,trading_n)
benchmark.frequency_optimizing()

--Frequency trading using MPT successfully performed--


<h2> 5. Conduct performance analysis </h2>

In [15]:
org = benchmark.frequency_weights[0][0]

In [ ]:
ben = org
exp = org*0.01
ret = np.array([1.023, 0.99, 0.983, 1.0023])

In [ ]:
# Create variables for we, wb, re, rb, Pe, Pb
# Then begin on gamma and Q. fml :)